In [ ]:
import hvplot.xarray  # noqa

`vectorfield` accepts 2d arrays of magnitude and angle on a grid and produces an array of vectors. x and y can be 2d or 1d coordinates. 

In [ ]:
import numpy as np
import xarray as xr
import geoviews as gv
from cartopy.examples import arrows

In [ ]:
xs, ys, U, V, crs = arrows.sample_data()

mag = np.sqrt(U**2 + V**2)
angle = (np.pi/2.) - np.arctan2(U/mag, V/mag)

ds = xr.Dataset({'mag': xr.DataArray(mag, dims=('y', 'x'), coords={'y': ys, 'x': xs}),
                 'angle': xr.DataArray(angle, dims=('y', 'x'), coords={'y': ys, 'x': xs})}, 
                attrs={'crs': crs})
ds

In [ ]:
ds.hvplot.vectorfield(x='x', y='y', angle='angle', mag='mag', hover=False).opts(magnitude='mag')

## Geographic Data
If a dataset has an attr called `crs` which is a cartopy object or a proj4 string, then just by setting the option `geo=True` will use the correct crs.

In [ ]:
ds.hvplot.vectorfield(x='x', y='y', angle='angle', mag='mag',
                      hover=False, geo=True).opts(magnitude='mag') * gv.tile_sources.CartoLight